<a href="https://colab.research.google.com/github/jay2jaykp/ML/blob/master/LogisticRegressionTimestemp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import pandas as pd
from collections import defaultdict, deque
from functools import partial
from tqdm import tqdm
#from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer


# import numpy as np
# from collections import defaultdict
# from functools import partial
# import matplotlib.pyplot as plt
# from matplotlib.pyplot import figure 
# import time
# import argparse
# 
# 
# #from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import MultiLabelBinarizer
# 

In [4]:
df = pd.read_csv('smalltrace.csv', sep='\t')
df.columns = ['no','timestamp','pid','pname','bno', 'bsize', 'op', 'dvmajor', 'dvminor', 'blockhash']
df.head()

,no,timestamp,pid,pname,bno,bsize,op,dvmajor,dvminor,blockhash
0,0,89966811921625,4257,nfsd,488846200,8,W,6,0,2c4787c7671c2f638777c4ab72859ce1
1,1,89967003980393,4253,nfsd,508455704,8,W,6,0,88b93b628d84082186026d9da044f173
2,2,89967004049935,4253,nfsd,508455712,8,W,6,0,b5e9f4e5ab62a4fff5313a606b0ad4e3
3,3,89967004093615,4253,nfsd,508455720,8,W,6,0,7af5556ad4d45263bcd8358867333dd4
4,4,89967324150054,4254,nfsd,508500288,8,W,6,0,88b93b628d84082186026d9da044f173


In [7]:
l = list(range(10))
df.size

100000

In [32]:
len(set(blocktrace))

9207

In [8]:
blocktrace = df['bno'].tolist()
sequences = list(range(df.size))
#convert element to int from str
blocktrace = np.array([int(x) for x in blocktrace])

In [9]:
''' 
  Input: Cache sequence [1, 2, 3]. get cache blocks by df['bno'][cache sequence #]
  get timestamp
  df['timestamp'][1], df['timestamp'][2], df['timestamp'][3]
  find avg. (avgTS)
  get relative => (avgTS - df['timestamp'][A])
  Output: [relativeA, relativeB, relativeC]
'''

def getRelativeTS(Cache_indices):
    tfsm=np.zeros((len(Cache_indices)))
    for i in Cache_indices:
        tfsm[Cache_indices.index(i)] = df.timestamp[i]

    avgTS = np.mean(tfsm)
    relTS = avgTS-tfsm

    return list(relTS)    
    #df.at[]

In [10]:
l=[0,1]
df.at[1,1]
#t=np.zeros((4,1))
t=np.array([1,2,3])
b=4
t=list(np.append(t,b))
t
np.mean([1,2,3])

ValueError: At based indexing on an non-integer index can only have non-integer indexers

In [0]:
  
''' 
  Input: Input: Cache sequence [1, 2, 3]. get cache blocks [A,B,C] by df['bno'][cache sequence #]
  gdaAbno
  df['bno'][1], df['bno'][2], df['bno'][3]
''''
def getRelativePos():

In [11]:


  

'''
get the furthest accessed block. Scans OPT dictionary and selects the maximum positioned element
'''

def getFurthestAccessBlock(C, OPT):
    maxAccessPosition = -1
    maxAccessBlock = -1
    for cached_block in C:
        if len(OPT[cached_block]) is 0:
            #print ( "Not Acccessing block anymore " + str(cached_block))
            return cached_block            
    for cached_block in C:  
        if OPT[cached_block][0] > maxAccessPosition:
            maxAccessPosition = OPT[cached_block][0]
            maxAccessBlock = cached_block
    #print ( "chose to evict " + str(maxAccessBlock) + " since its position of access is " + str(maxAccessPosition))
    return maxAccessBlock

def belady_opt(blocktrace,sequences,frame):
    OPT = defaultdict(partial(np.ndarray,0))

    for i, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        OPT[block] = np.append(OPT[block], i)    

    #print ("created OPT dictionary")    

    hit, miss = 0, 0

    C = deque()
    C2 = deque()
    d = defaultdict(deque)
    d_ftime = set()
    d_timestamp = {}
    d_label = {}
    
    for k,block in enumerate(tqdm(blocktrace, desc="OPT", leave=False)):

        if block in C:
            #OPT[block] = OPT[block][1:]
            OPT[block] = np.delete(OPT[block],0)
            hit+=1
            #print('hit' + str(block))
            #print(OPT)
        else:
            #print('miss' + str(block))
            miss+=1
            if len(C) == frame:
                fblock = getFurthestAccessBlock(C, OPT)
                assert(fblock != -1)
                d[fblock] = deque(zip(C,C2)) 
                d_timestamp[fblock] = getRelativeTS(list(C2)) #Returning Time Stemp
                d_label[fblock] = C.index(fblock)
                C2.remove(C2[C.index(fblock)])
                C.remove(fblock)
                #C2.remove()
                d_ftime.add(sequences[k])
            C.append(block)
            C2.append(k)
            d_ftime.add(sequences[k])
            #OPT[block] = OPT[block][1:]
            #print(OPT)
            OPT[block] = np.delete(OPT[block],[0])

    #print ("hit count" + str(hit_count))
    #print ("miss count" + str(miss_count))
    hitrate = hit / (hit + miss)
    print(hitrate)
    return d,d_timestamp,d_label

In [47]:
d,d_timestamp,d_label = belady_opt(blocktrace,sequences, 500)

OPT: building index: 100%|███████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 216683.75it/s]


0.0793


In [38]:
d[14254312]

deque([(14254312, 7130),
       (485954192, 7317),
       (485954200, 7318),
       (486242128, 7323),
       (486242136, 7324),
       (486242144, 7325),
       (485954208, 7328),
       (485954216, 7329),
       (485954224, 7330),
       (485954232, 7331),
       (283459936, 7547),
       (283459944, 7548),
       (283459952, 7549),
       (283459960, 7550),
       (283459968, 7551),
       (283459976, 7552),
       (283459984, 7553),
       (283459992, 7554),
       (283460000, 7555),
       (283460008, 7556),
       (283460016, 7557),
       (283460024, 7558),
       (283460032, 7559),
       (283460040, 7560),
       (283460048, 7561),
       (283460056, 7562),
       (283460064, 7563),
       (283460072, 7564),
       (283460080, 7565),
       (283460088, 7566),
       (283460096, 7567),
       (283460104, 7568),
       (283460112, 7569),
       (283460120, 7570),
       (283460128, 7571),
       (283460136, 7572),
       (283460144, 7573),
       (283460152, 7574),
       (28346

In [39]:
len(set(l))

9197

In [48]:
#Time Stamp only (Training data)
data = []
for i,j in d_timestamp.items():
    dat = list(map(str, j))
    data.append(' '.join(dat))
    #label.append(i)
len(data)

8707

In [49]:
#Time Stamp only (Label data)
label = []
for i,j in d_label.items():
    label.append(j)
len(set(label))

16

In [50]:
#CountVEctorizer for fitting the data
vec = CountVectorizer(binary=True)
vec_ = vec.fit(data)
DATA = vec_.transform(data)
DATA

<8707x4288746 sparse matrix of type '<class 'numpy.int64'>'
	with 4338246 stored elements in Compressed Sparse Row format>

In [51]:
#Train-Test split
X_train, X_test, Y_train, Y_test = train_test_split(DATA, label ,test_size=0.5, shuffle=True)

In [52]:
#Fitting Logistic Regression Model
clf = LogisticRegression().fit(X_train, Y_train)

In [53]:
#Accuracy Testing
accuracy_score(Y_test, clf.predict(X_test))

0.288929719797887

In [54]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, clf.predict(X_test))

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,  344],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    5],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,   92],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    3],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    5],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,  267],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,  436],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,  687],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,  326],
       [   0,    0,    0,    0,    0,